## 2nd variant
### Dynamic Bayesian Network Structure Learning with variational circuit.
This is a toy example with dummy generated training data of 4 variables, that means 8 vertices (4 for t and 4 for t+1).    
n_qubits=6  
where first three digits correspond to vertice where the edge begins and three las digits correspond to vertice qhere edge ends i.e. 010101 corresponds to: vertice_2 ---> vertice_5  

It was tested with two sets of test data at the end of the notebook. The last test is the mean of 7 experiments of the same network, where each experiment is build with data for t and t+1  


In [1]:
import pandas as pd
from torch.utils.data import Dataset
import torch
import torchvision
from torch import nn
import numpy as np
import pennylane as qml
import random

In [2]:
path = "tiny_dummy_qhack.csv"

In [3]:
ds = pd.read_csv(path)
ds

,vertices,experiment,t0,t1
0,v0,1,0.50,0.10
1,v1,1,0.03,0.45
2,v2,1,0.50,0.04
3,v3,1,0.60,0.80
4,v0,2,0.10,0.07
5,v1,2,0.45,0.10
6,v2,2,0.04,0.04
7,v3,2,0.80,0.50
8,v0,3,0.07,0.05
9,v1,3,0.10,0.06


In [4]:
def get_edges(n=4):
    num_edges = random.randint(n, n+3)
    e1 = [(random.randint(0, n-1),random.randint(0, (n*2)-1)) for f in range(num_edges//2)]
    e2 = [(random.randint(0, (n*2)-1),random.randint(n, (n*2)-1)) for f in range(num_edges//2)]
    return e1 + e2

def get_t0(edges, n=4):
    t0 = np.zeros(n) + 0.01
    edges0 = [edge for i in range(n) for edge in edges if edge[0] == i and edge[1] < n]
    if len(edges0) > 0:
        t0[edges0[0][0]] = random.random()
        for edge in edges0:
            t0[edge[1]] += t0[edge[0]]
    return t0
        
def get_t1(edges, t0, n=4):
    t1 = np.zeros(n) + 0.01
    edges1 = [edge for edge in edges if edge[1] >= n]
    for edge in edges1:
        if edge[0] < n:
            t1[edge[1]-n] += t0[edge[0]]
        else:
            t1[edge[1]-n] += t1[edge[0]-n]
    return t1

In [5]:
# generate training dataset
exper = 300
nnodes = 4
n_qubits = 6
arr_list = []
edges_list = []
for f in range(exper):
    edges = get_edges(n = nnodes)
    t0 = get_t0(edges, n = nnodes)
    t1 = get_t1(edges, t0, n = nnodes)
    arr_list.append(np.stack([t0,t1]))
    edges_list.append(edges)
arr = np.concatenate(arr_list, axis=1)

In [6]:
dsa = pd.DataFrame({'t01':arr_list})
dsa

,t01
0,"[[0.01, 0.01, 0.01, 0.01], [0.03, 0.02, 0.06, ..."
1,"[[0.871966394677728, 0.881966394677728, 0.01, ..."
2,"[[0.01, 0.01, 0.01, 0.3253832205503371], [0.04..."
3,"[[0.01, 0.47521740044959204, 0.01, 0.465217400..."
4,"[[0.08307927543786242, 0.02, 0.01, 0.01], [0.0..."
...,...
295,"[[0.01, 0.01, 0.3146875664939037, 0.01], [0.01..."
296,"[[0.01, 0.01, 0.8401179651259245, 0.01], [0.04..."
297,"[[0.06523514115080853, 0.08523514115080852, 0...."
298,"[[0.01, 0.278502448859859, 0.26850244885985897..."


In [7]:
#int("110100010",2) = 418
edges_bin_list = [[np.binary_repr(ed[0], width=n_qubits//2) + np.binary_repr(ed[1], width=n_qubits//2)  for ed in edges] for edges in edges_list]
ya_list = [[int(edge,2) for edge in edges] for edges in edges_bin_list]

In [8]:
dsa['y'] = ya_list

In [9]:
dsa

,t01,y
0,"[[0.01, 0.01, 0.01, 0.01], [0.03, 0.02, 0.06, ...","[28, 30, 6, 60, 54, 61]"
1,"[[0.871966394677728, 0.881966394677728, 0.01, ...","[1, 21, 14, 52, 45, 55]"
2,"[[0.01, 0.01, 0.01, 0.3253832205503371], [0.04...","[27, 4, 21, 31, 29, 36]"
3,"[[0.01, 0.47521740044959204, 0.01, 0.465217400...","[25, 20, 37, 52]"
4,"[[0.08307927543786242, 0.02, 0.01, 0.01], [0.0...","[0, 25, 30, 21, 31, 14]"
...,...,...
295,"[[0.01, 0.01, 0.3146875664939037, 0.01], [0.01...","[14, 18, 23, 39]"
296,"[[0.01, 0.01, 0.8401179651259245, 0.01], [0.04...","[18, 29, 12, 44]"
297,"[[0.06523514115080853, 0.08523514115080852, 0....","[2, 17, 6, 60]"
298,"[[0.01, 0.278502448859859, 0.26850244885985897...","[17, 12, 4, 14]"


In [89]:
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode2(inputs, eweights, weights):
    for i in range(nnodes):
        qml.Rot(inputs[i], inputs[i+4], inputs[i+4]-inputs[i], wires=i)
    for W in weights:
            for i in range(n_qubits):
                qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)

    qml.BasicEntanglerLayers(eweights, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))

In [90]:
n_layers = 4
weight_shapes = {"weights": (n_layers, n_qubits, 3), "eweights": (n_layers, n_qubits)}

In [91]:
qlayer2 = qml.qnn.TorchLayer(qnode2, weight_shapes)

In [92]:
layers = [qlayer2]
model = torch.nn.Sequential(*layers)

In [93]:
#optimizer = torch.optim.SGD(model.parameters(), lr=0.2)
#criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

In [94]:
class CustomDataset(Dataset):
    def __init__(self, ds, n, q, transform=None):
        self.ds_full = ds
        self.n = n
        self.q = q
        self.x_csv = self.ds_full[["t01"]]
        self.y_csv = self.ds_full[["y"]]
        self.transform = transform

    def __len__(self):
        return len(self.x_csv)

    def __getitem__(self, idx):
        x = np.array(self.x_csv.iloc[idx].tolist())
        y = np.zeros(2**self.q)
        for i in self.y_csv.iloc[idx].tolist():
            #011000 24
            y[i] = 1
        if self.transform:
            x = self.transform(x)
            y = self.transform(y)
        return x, y

In [95]:
batch_size = 1
transform = torchvision.transforms.Lambda(lambda y: torch.from_numpy(y).float())

train_dataset = CustomDataset(dsa, nnodes, n_qubits, transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True
)

test_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False
)

In [96]:
epochs=30
input_size=nnodes*2
for epoch in range(epochs):
    loss = 0
    for batch_features, y_batch in train_loader:
        batch_features = batch_features.view(-1, input_size)
        
        optimizer.zero_grad()
        
        outputs = model(batch_features)
        
        train_loss = criterion(outputs, y_batch)
        
        train_loss.backward()
        
        optimizer.step()
        
        loss += train_loss.item()
    
    loss = loss / len(train_loader)
    
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch : 1/30, loss = 0.072116
epoch : 2/30, loss = 0.071765
epoch : 3/30, loss = 0.071534
epoch : 4/30, loss = 0.071436
epoch : 5/30, loss = 0.071380
epoch : 6/30, loss = 0.071344
epoch : 7/30, loss = 0.071313
epoch : 8/30, loss = 0.071285
epoch : 9/30, loss = 0.071266
epoch : 10/30, loss = 0.071249
epoch : 11/30, loss = 0.071231
epoch : 12/30, loss = 0.071217
epoch : 13/30, loss = 0.071209
epoch : 14/30, loss = 0.071198
epoch : 15/30, loss = 0.071190
epoch : 16/30, loss = 0.071181
epoch : 17/30, loss = 0.071171
epoch : 18/30, loss = 0.071163
epoch : 19/30, loss = 0.071149
epoch : 20/30, loss = 0.071139
epoch : 21/30, loss = 0.071128
epoch : 22/30, loss = 0.071112
epoch : 23/30, loss = 0.071097
epoch : 24/30, loss = 0.071080
epoch : 25/30, loss = 0.071064
epoch : 26/30, loss = 0.071047
epoch : 27/30, loss = 0.071025
epoch : 28/30, loss = 0.071011
epoch : 29/30, loss = 0.070990
epoch : 30/30, loss = 0.070978


In [41]:
batch_features

tensor([[0.3756, 0.0100, 0.3656, 0.0200, 0.3856, 0.0100, 0.0200, 0.3956]])

In [98]:
outputs = model(batch_features)

In [99]:
outputs

tensor([[1.0025e-02, 1.6791e-02, 8.4450e-03, 2.3464e-03, 1.8715e-02, 7.2561e-02,
         1.9484e-02, 6.9921e-02, 6.7758e-03, 4.9412e-03, 2.9542e-03, 7.3887e-03,
         3.2827e-02, 2.9133e-02, 3.9609e-02, 7.7445e-02, 1.5848e-02, 6.7690e-03,
         2.7286e-03, 7.8136e-03, 7.1204e-02, 4.3367e-02, 3.6880e-02, 3.6641e-02,
         2.3232e-03, 1.0489e-02, 1.0758e-02, 1.2972e-02, 6.7464e-02, 2.4259e-02,
         7.2436e-02, 2.3316e-02, 2.1746e-03, 6.0891e-05, 7.3948e-04, 1.5079e-03,
         2.3120e-03, 4.8731e-03, 3.9451e-03, 1.3496e-02, 3.0727e-05, 6.7232e-05,
         1.0649e-03, 1.2499e-03, 2.9433e-03, 1.4062e-02, 1.7184e-03, 9.3774e-03,
         1.0152e-03, 1.2367e-03, 5.1341e-04, 2.2311e-04, 1.4927e-02, 2.6407e-03,
         1.4752e-02, 6.1181e-03, 1.2985e-03, 9.2424e-04, 1.0012e-04, 1.5904e-03,
         1.2332e-02, 6.5080e-03, 9.2857e-03, 2.2820e-03]],
       grad_fn=<StackBackward0>)

In [100]:
np.flip(np.argsort(outputs.detach().numpy()))

array([[15,  5, 30, 20,  7, 28, 21, 14, 22, 23, 12, 13, 29, 31,  6,  4,
         1, 16, 52, 54, 45, 39, 27, 60, 26, 25,  0, 47, 62,  2, 19, 11,
         8, 17, 61, 55,  9, 37, 38, 10, 44, 18, 53,  3, 24, 36, 63, 32,
        46, 59, 35, 56, 43, 49, 42, 48, 57, 34, 50, 51, 58, 41, 33, 40]])

In [101]:
np.nonzero(y_batch)

tensor([[ 0,  8],
        [ 0, 16],
        [ 0, 21],
        [ 0, 31],
        [ 0, 45],
        [ 0, 52]])

## testing1

In [102]:
path = "tiny_dummy_qhack_test.csv"

In [103]:
dstemp = pd.read_csv(path)
dstemp

,vertices,experiment,t0,t1
0,v0,1,0.060,0.060
1,v1,1,0.070,0.070
2,v2,1,0.500,0.065
3,v3,1,0.060,0.450
4,v0,2,0.060,0.050
5,v1,2,0.070,0.060
6,v2,2,0.065,0.060
7,v3,2,0.450,0.060
8,v0,3,0.050,0.050
9,v1,3,0.060,0.060


In [104]:
t01_list = [dstemp[['t0','t1']].iloc[f*4:(f+1)*4].values.T for f in range(4)]
y_list = [[8,25,7,6] for i in range(4)]
dst = pd.DataFrame({'t01':t01_list,'y':y_list})
dst

,t01,y
0,"[[0.06, 0.07, 0.5, 0.06], [0.06, 0.07, 0.065, ...","[8, 25, 7, 6]"
1,"[[0.06, 0.07, 0.065, 0.45], [0.05, 0.06, 0.06,...","[8, 25, 7, 6]"
2,"[[0.05, 0.06, 0.06, 0.06], [0.05, 0.06, 0.05, ...","[8, 25, 7, 6]"
3,"[[0.05, 0.06, 0.05, 0.05], [0.05, 0.06, 0.05, ...","[8, 25, 7, 6]"


In [105]:
n = 1
transform = torchvision.transforms.Lambda(lambda y: torch.from_numpy(y).float())

test_dataset = CustomDataset(dst, nnodes, n_qubits, transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True
)


In [106]:
experiments = []
outputs_list = []
for batch_features, _ in test_loader:
    batch_features = batch_features.view(-1, input_size)
    batch_features
    outputs = model(batch_features)
    outputs_list.append(outputs)
    experiments.append(np.flip(np.argsort(outputs.detach().numpy())))
experiments

[array([[15,  7, 20, 28,  5, 30, 21, 14, 23, 12, 22, 13, 29, 31,  9, 45,
         54, 52,  6,  4, 39,  1, 60, 18, 26, 17, 62, 47, 19, 24,  8, 61,
         16, 27,  3, 55, 37, 10, 38, 25,  0, 44, 53, 63, 36, 11, 32,  2,
         57, 46, 59, 34, 33, 58, 35, 49, 56, 42, 50, 48, 41, 40, 51, 43]]),
 array([[15, 30,  5, 20,  7, 28, 21, 14, 23, 22, 12, 13, 29, 31,  6,  4,
         16, 54, 52, 45,  1, 39, 27, 60, 25, 26, 62,  0, 47,  2, 19, 11,
         17, 55,  8, 61,  9, 37, 38,  3, 44, 24, 53, 10, 18, 32, 63, 46,
         36, 59, 57, 49, 35, 42, 56, 43, 34, 48, 50, 58, 33, 51, 41, 40]]),
 array([[15, 30,  5, 20,  7, 28, 21, 14, 23, 22, 12, 13, 29, 31,  6,  4,
         16,  1, 54, 52, 45, 39, 27, 60, 26, 25, 62,  0, 47,  2, 19, 17,
         11, 55,  8, 61,  9, 37, 38, 44, 53,  3, 10, 18, 24, 32, 63, 36,
         46, 59, 35, 49, 57, 56, 42, 43, 34, 48, 50, 33, 58, 51, 41, 40]]),
 array([[15, 30,  5, 20,  7, 28, 21, 14, 23, 22, 12, 13, 29, 31,  6,  4,
         16,  1, 54, 52, 45, 39, 27, 60, 2

In [107]:
results = np.flip(np.argsort(sum(outputs_list).detach().numpy()))

In [108]:
np.max(sum(outputs_list).detach().numpy()),np.min(sum(outputs_list).detach().numpy())

(0.31877974, 0.00015223661)

In [109]:
sum(outputs_list)[0][25]

tensor(0.0346, grad_fn=<SelectBackward0>)

In [110]:
results_bin = [np.binary_repr(f, width=n_qubits) for f in results.tolist()[0]]

In [111]:
results_bin[:12]

['001111',
 '010100',
 '000111',
 '000101',
 '011110',
 '011100',
 '010101',
 '001110',
 '010111',
 '010110',
 '001100',
 '001101']

In [113]:
results.tolist()[0][:12], [8,25,7,6]

([15, 20, 7, 5, 30, 28, 21, 14, 23, 22, 12, 13], [8, 25, 7, 6])

## testing2

In [114]:
ds

,vertices,experiment,t0,t1
0,v0,1,0.50,0.10
1,v1,1,0.03,0.45
2,v2,1,0.50,0.04
3,v3,1,0.60,0.80
4,v0,2,0.10,0.07
5,v1,2,0.45,0.10
6,v2,2,0.04,0.04
7,v3,2,0.80,0.50
8,v0,3,0.07,0.05
9,v1,3,0.10,0.06


In [115]:
t01_list = [ds[['t0','t1']].iloc[f*4:(f+1)*4].values.T for f in range(7)]
y_list = [[24,19,5,15,23] for i in range(7)]
dst2 = pd.DataFrame({'t01':t01_list,'y':y_list})
dst2

,t01,y
0,"[[0.5, 0.03, 0.5, 0.6], [0.1, 0.45, 0.04, 0.8]]","[24, 19, 5, 15, 23]"
1,"[[0.1, 0.45, 0.04, 0.8], [0.07, 0.1, 0.04, 0.5]]","[24, 19, 5, 15, 23]"
2,"[[0.07, 0.1, 0.04, 0.5], [0.05, 0.06, 0.03, 0....","[24, 19, 5, 15, 23]"
3,"[[0.05, 0.06, 0.03, 0.15], [0.04, 0.04, 0.03, ...","[24, 19, 5, 15, 23]"
4,"[[0.04, 0.04, 0.03, 0.1], [0.04, 0.04, 0.03, 0...","[24, 19, 5, 15, 23]"
5,"[[0.04, 0.04, 0.03, 0.07], [0.04, 0.04, 0.03, ...","[24, 19, 5, 15, 23]"
6,"[[0.04, 0.04, 0.03, 0.07], [0.04, 0.04, 0.03, ...","[24, 19, 5, 15, 23]"


In [116]:
n = 1
transform = torchvision.transforms.Lambda(lambda y: torch.from_numpy(y).float())

test_dataset = CustomDataset(dst2, nnodes, n_qubits, transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True
)


In [117]:
experiments = []
outputs_list = []
for batch_features, _ in test_loader:
    batch_features = batch_features.view(-1, input_size)
    batch_features
    outputs = model(batch_features)
    outputs_list.append(outputs)
    experiments.append(np.flip(np.argsort(outputs.detach().numpy())))
experiments

[array([[ 7, 15, 28, 20, 30,  5, 21, 14, 23, 12,  9, 22, 13, 18, 29, 24,
         45, 31, 17, 54,  3, 16, 39, 52, 60,  6, 19,  4,  8, 10, 61, 62,
         55, 33,  1, 35, 26, 11, 47, 56, 42, 53, 58, 48, 40, 44, 38, 49,
         63, 46, 51, 37, 36, 27, 43, 41, 50, 57, 25, 34, 32, 59,  2,  0]]),
 array([[15, 20, 30,  7,  5, 28, 21, 14, 23, 12, 22, 29, 13, 31, 45, 54,
          6, 16, 52,  4, 39,  9, 24, 60,  3, 62,  1, 47, 18, 11, 61, 17,
         26, 27, 55, 19, 25, 37,  8, 38,  2,  0, 53, 10, 63, 44, 33, 36,
         58, 46, 42, 49, 32, 48, 59, 57, 34, 50, 43, 41, 51, 40, 56, 35]]),
 array([[15, 20, 30,  5,  7, 28, 21, 14, 23, 22, 12, 13, 29, 31,  6,  4,
         16, 54, 45, 52,  1, 39, 60, 27, 62, 26, 25, 47,  0, 19, 11, 17,
          2, 61,  9, 55,  8, 37,  3, 24, 38, 18, 44, 53, 63, 10, 36, 46,
         32, 59, 49, 42, 57, 43, 35, 34, 56, 48, 50, 33, 58, 51, 41, 40]]),
 array([[15, 20, 30,  5,  7, 28, 21, 14, 23, 22, 12, 13, 29, 31,  6,  4,
         54, 52,  1, 16, 45, 39, 60, 27, 2

In [118]:
results = np.flip(np.argsort(sum(outputs_list).detach().numpy()))

In [119]:
np.max(sum(outputs_list).detach().numpy()),np.min(sum(outputs_list).detach().numpy())

(0.5703525, 0.0016959505)

In [120]:
results_bin = [np.binary_repr(f, width=n_qubits) for f in results.tolist()[0]]

In [121]:
sum(outputs_list)[0][5]

tensor(0.5083, grad_fn=<SelectBackward0>)

In [125]:
results.tolist()[0][:12],[24,19,5,15,23]

([15, 20, 7, 30, 5, 28, 21, 14, 23, 22, 12, 13], [24, 19, 5, 15, 23])

In [127]:
results_bin[:12], [np.binary_repr(f, width=n_qubits) for f in [24,19,5,15,23]]

(['001111',
  '010100',
  '000111',
  '011110',
  '000101',
  '011100',
  '010101',
  '001110',
  '010111',
  '010110',
  '001100',
  '001101'],
 ['011000', '010011', '000101', '001111', '010111'])